In [1]:
'''
使用的数据可以从以下网址下载：
https://www.kaggle.com/c/dogs-vs-cats/data
在我们的设置中，我们：
- 创建一个 data/ 文件夹
- 在 data/ 里面 创建 train/ and validation/ 子文件夹
- 在 train/ 和 validation/ 里面 创建 cats/ and dogs/ 子文件夹
- 将猫图片索引0-999放入data/train/cats中
- 将猫图片索引1000-1400放在data/validation/cats中
- 把狗图片索引12500-13499放在data/train/dogs中
- 将狗图片索引13500-13900放在data/validation/dogs中
因此，我们每个类有1000个训练范例，每个类有400个验证范例。
总之，这是我们的目录结构：
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense


# 我们的图像的尺寸。
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# 构建VGG16网络

base_model = applications.VGG16(weights='imagenet',
                                include_top=False,
                                input_shape=(img_width, img_height, 3))
print('Model loaded.')

# 在卷积模型的基础上建立分类器模型
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# 注意，为了成功地进行微调，有必要从一个完全训练的分类器开始，包括顶级分类器

# 在基础卷积上加上模型
model = Sequential()
model.add(base_model)
model.add(top_model)
# 设置前25层（直到最后一个conv块）
# 不可训练（重量不更新）
for layer in model.layers[:25]:
    layer.trainable = False

# 用SGD/momentum优化器和非常慢的学习速度编译模型。
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# 准备数据扩充配置
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height,
                                                                 img_width),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# 微调模型
model.fit_generator(train_generator,
                    samples_per_epoch=nb_train_samples,
                    epochs=epochs,
                    validation_data=validation_generator,
                    nb_val_samples=nb_validation_samples)
model.save_weights('try_2.h5')

Using TensorFlow backend.
D:\ProgramData\Anaconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\envs\deep\lib\site-packag


Model loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.

Epoch 1/50

D:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:109: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
D:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:109: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., steps_per_epoch=125, validation_steps=800)`



125/125 [==============================] - 156s 1s/step - loss: 0.7787 - accuracy: 0.5325 - val_loss: 0.5957 - val_accuracy: 0.5475
Epoch 2/50
125/125 [==============================] - 158s 1s/step - loss: 0.7577 - accuracy: 0.5420 - val_loss: 0.6802 - val_accuracy: 0.5475
Epoch 3/50
125/125 [==============================] - 158s 1s/step - loss: 0.7692 - accuracy: 0.5250 - val_loss: 0.6473 - val_accuracy: 0.5475
Epoch 4/50
125/125 [==============================] - 159s 1s/step - loss: 0.7703 - accuracy: 0.5355 - val_loss: 0.5258 - val_accuracy: 0.5475
Epoch 5/50
125/125 [==============================] - 162s 1s/step - loss: 0.7719 - accuracy: 0.5290 - val_loss: 0.5576 - val_accuracy: 0.5475
Epoch 6/50
125/125 [==============================] - 162s 1s/step - loss: 0.7651 - accuracy: 0.5375 - val_loss: 0.6055 - val_accuracy: 0.5475
Epoch 7/50
124/125 [============================>.] - ETA: 0s - loss: 0.7647 - accuracy: 0.5338

KeyboardInterrupt: 